In [1]:
import pandas as pd

In [2]:
# carregar os dados do csv para um dataframe
df = pd.read_csv("fifa_world_cup_matches.csv")

In [3]:
display(df.head())

display('columns:', df.columns)

print()
df.info()

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930


'columns:'

Index(['home', 'score', 'away', 'year'], dtype='object')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   home    929 non-null    object
 1   score   929 non-null    object
 2   away    929 non-null    object
 3   year    929 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 29.2+ KB


In [6]:
# Limpeza e Preparação dos Dados

# Primeiro eliminar linha onde o placar e Cancelled
df = df.drop(df[df["score"] == "Cancelled"].index)

# depois precisamos dividir a coluna score em duas colunas separadas para os gols dos times da casa e visitantes.

# Dividir a coluna 'score' em 'home_goals' e 'away_goals'
df[["home_goals", "away_goals"]] = df["score"].str.split("–", expand=True)

# extrair os placares e transformar em int
df["home_goals"] = df["home_goals"].str.extract("(\\d+)").astype(int)
df["away_goals"] = df["away_goals"].str.extract("(\\d+)").astype(int)

# Remover espaços extras nas colunas 'home' e 'away'
df["home"] = df["home"].str.strip()
df["away"] = df["away"].str.strip()

# Determinar o vencedor de cada partida
df["winner"] = df.apply(
    lambda row: (
        row["home"]
        if row["home_goals"] > row["away_goals"]
        else (row["away"] if row["away_goals"] > row["home_goals"] else "Empate")
    ),
    axis=1,
)

#exemplo de analises

In [14]:
# Contar o número de vitórias de cada país
vitorias = df[df["winner"] != "Empate"]["winner"].value_counts()
print(vitorias)

winner
Brazil                    73
Argentina                 46
Italy                     42
France                    39
West Germany              34
                          ..
Jamaica                    1
Czech Republic             1
Slovenia                   1
Slovakia                   1
Bosnia and Herzegovina     1
Name: count, Length: 65, dtype: int64


In [12]:
df

,home,score,away,year,home_goals,away_goals,winner
0,France,4–1,Mexico,1930,4,1,France
1,Argentina,1–0,France,1930,1,0,Argentina
2,Chile,3–0,Mexico,1930,3,0,Chile
3,Chile,1–0,France,1930,1,0,Chile
4,Argentina,6–3,Mexico,1930,6,3,Argentina
...,...,...,...,...,...,...,...
924,England,1–2,France,2022,1,2,France
925,Argentina,3–0,Croatia,2022,3,0,Argentina
926,France,2–0,Morocco,2022,2,0,France
927,Croatia,2–1,Morocco,2022,2,1,Croatia


In [7]:
df

,home,score,away,year,home_goals,away_goals
0,France,4–1,Mexico,1930,4,1
1,Argentina,1–0,France,1930,1,0
2,Chile,3–0,Mexico,1930,3,0
3,Chile,1–0,France,1930,1,0
4,Argentina,6–3,Mexico,1930,6,3
...,...,...,...,...,...,...
924,England,1–2,France,2022,1,2
925,Argentina,3–0,Croatia,2022,3,0
926,France,2–0,Morocco,2022,2,0
927,Croatia,2–1,Morocco,2022,2,1


In [19]:
# Calcular a média de gols por jogo
media_gols = (df["home_goals"] + df["away_goals"]).mean()
print(f"Média de gols por jogo: {media_gols:.1f}")

Média de gols por jogo: 2.8


In [20]:
# Calcular o número total de gols por ano
gols_por_ano = df.groupby("year").apply(
    lambda x: x["home_goals"].sum() + x["away_goals"].sum()
)
print(gols_por_ano)

year
1930     70
1934     70
1938     84
1950     88
1954    140
1958    126
1962     89
1966     89
1970     95
1974     97
1978    102
1982    146
1986    132
1990     33
1994    141
1998    171
2002    161
2006    147
2010    145
2014    171
2018    169
2022    172
dtype: int64


C:\Users\andre\AppData\Local\Temp\ipykernel_9412\1972909103.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gols_por_ano = df.groupby("year").apply(


In [21]:
# Contar o número total de gols marcados por cada time
gols_por_time_home = df.groupby("home")["home_goals"].sum()
gols_por_time_away = df.groupby("away")["away_goals"].sum()
gols_por_time = gols_por_time_home.add(gols_por_time_away, fill_value=0).sort_values(
    ascending=False
)
print(gols_por_time)

Brazil                 233.0
Argentina              149.0
France                 136.0
Italy                  124.0
West Germany           121.0
                       ...  
Israel                   1.0
China                    0.0
Dutch East Indies        0.0
Trinidad and Tobago      0.0
Zaire                    0.0
Length: 85, dtype: float64


In [26]:
# Número de vitórias, derrotas e empates por ano
resultados_por_ano = df.groupby("year")["winner"].value_counts().unstack().fillna('-')
print(resultados_por_ano)

winner Algeria Argentina Australia Austria Belgium Bosnia and Herzegovina  \
year                                                                        
1930         -       4.0         -       -       -                      -   
1934         -         -         -     2.0       -                      -   
1938         -         -         -       -       -                      -   
1950         -         -         -       -       -                      -   
1954         -         -         -     4.0       -                      -   
1958         -       1.0         -       -       -                      -   
1962         -       1.0         -       -       -                      -   
1966         -       2.0         -       -       -                      -   
1970         -         -         -       -     1.0                      -   
1974         -       1.0         -       -       -                      -   
1978         -       5.0         -     3.0       -                      -   